In [ ]:
!pip install kaggle



In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'jeffwalker24'
os.environ['KAGGLE_KEY'] = 'your_kaggle_api_key'


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jeffwalker24","key":"0c81df049a87d48bad7fbb2e90303bc0"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d Cornell-University/arxiv



Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv
License(s): CC0-1.0
100% 1.32G/1.32G [00:29<00:00, 81.4MB/s]
100% 1.32G/1.32G [00:29<00:00, 48.3MB/s]


In [ ]:
!unzip arxiv.zip -d ./arxiv_data



Archive:  arxiv.zip
  inflating: ./arxiv_data/arxiv-metadata-oai-snapshot.json  


In [ ]:
import pandas as pd
import json
from datetime import datetime

def process_chunk(chunk):
    # Select only the necessary columns
    chunk = chunk[['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no',
                   'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed']]

    # Extract latest version
    chunk['main_version'] = chunk['versions'].apply(lambda x: json.loads(x)[-1] if isinstance(x, str) else x[-1])

    # Convert date
    chunk['date'] = chunk['main_version'].apply(lambda x: datetime.strptime(x['created'], '%a, %d %b %Y %H:%M:%S GMT'))

    # Drop unnecessary columns
    chunk = chunk.drop(['versions', 'main_version'], axis=1)

    return chunk

# Define the chunk size (number of rows to process at a time)
chunk_size = 100  # Adjust based on your available memory

# Open the output file in append mode
with open('preprocessed_arxiv_data.csv', 'w') as f:
    header_written = False

    # Read and process the JSON file in very small chunks
    for chunk in pd.read_json('./arxiv_data/arxiv-metadata-oai-snapshot.json', lines=True, chunksize=chunk_size):
        # Process the chunk
        processed_chunk = process_chunk(chunk)

        # Write the chunk to the CSV file
        processed_chunk.to_csv(f, index=False, header=not header_written, mode='a')

        # Update header_written flag
        if not header_written:
            header_written = True

        # Clear memory
        del processed_chunk

    print("Preprocessing complete and data saved to 'preprocessed_arxiv_data.csv'.")

Preprocessing complete and data saved to 'preprocessed_arxiv_data.csv'.


In [ ]:
# scripts/load_data.py
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_and_split_data(file_path='project/data/preprocessed_arxiv_data.csv', test_size=0.2, val_size=0.1, random_state=42):
    """
    Load data from a CSV file and split it into train, validation, and test sets.

    Args:
    file_path (str): Path to the CSV file
    test_size (float): Proportion of the dataset to include in the test split
    val_size (float): Proportion of the training data to include in the validation split
    random_state (int): Seed for random number generator for reproducibility

    Returns:
    tuple: (train, val, test) DataFrames
    """
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"The file {file_path} does not exist.")

        # Load the data
        logging.info(f"Loading data from {file_path}")
        df = pd.read_csv(file_path)

        # Check if DataFrame is empty
        if df.empty:
            raise ValueError("The loaded DataFrame is empty.")

        # Log basic information about the dataset
        logging.info(f"Loaded {len(df)} rows and {len(df.columns)} columns")

        # Perform the split
        logging.info("Splitting data into train, validation, and test sets")
        train, test = train_test_split(df, test_size=test_size, random_state=random_state)
        train, val = train_test_split(train, test_size=val_size / (1 - test_size), random_state=random_state)

        # Log split sizes
        logging.info(f"Train set: {len(train)} rows")
        logging.info(f"Validation set: {len(val)} rows")
        logging.info(f"Test set: {len(test)} rows")

        return train, val, test

    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        # Update the file_path to the correct location of the CSV file
        train_data, val_data, test_data = load_and_split_data(file_path='/content/preprocessed_arxiv_data.csv')
        logging.info("Data successfully loaded and split into train, validation, and test sets.")
    except Exception as e:
        logging.error(f"Failed to load and split data: {str(e)}")


<ipython-input-1-36abfaf0d0f4>:30: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_and_split_data(file_path='/content/preprocessed_arxiv_data.csv', chunk_size=10000, test_size=0.2, val_size=0.1, random_state=42):
    """
    Load data from a CSV file in chunks and split it into train, validation, and test sets.

    Args:
    file_path (str): Path to the CSV file
    chunk_size (int): Number of rows per chunk
    test_size (float): Proportion of the dataset to include in the test split
    val_size (float): Proportion of the training data to include in the validation split
    random_state (int): Seed for random number generator for reproducibility

    Returns:
    tuple: (train, val, test) DataFrames
    """
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"The file {file_path} does not exist.")

        # Initialize empty lists to store train, val, and test data
        train_chunks = []
        val_chunks = []
        test_chunks = []

        # Process the file in chunks
        for chunk in pd.read_csv(file_path, chunksize=chunk_size):
            # Perform a train/test split on each chunk
            train, test = train_test_split(chunk, test_size=test_size, random_state=random_state)
            train, val = train_test_split(train, test_size=val_size / (1 - test_size), random_state=random_state)

            # Append the split data to the respective lists
            train_chunks.append(train)
            val_chunks.append(val)
            test_chunks.append(test)

        # Concatenate the chunks into final DataFrames
        train_df = pd.concat(train_chunks, ignore_index=True)
        val_df = pd.concat(val_chunks, ignore_index=True)
        test_df = pd.concat(test_chunks, ignore_index=True)

        # Log the final sizes
        logging.info(f"Final Train set: {len(train_df)} rows")
        logging.info(f"Final Validation set: {len(val_df)} rows")
        logging.info(f"Final Test set: {len(test_df)} rows")

        return train_df, val_df, test_df

    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        # Update the file_path to the correct location of the CSV file
        train_data, val_data, test_data = load_and_split_data(file_path='/content/preprocessed_arxiv_data.csv')
        logging.info("Data successfully loaded and split into train, validation, and test sets.")
    except Exception as e:
        logging.error(f"Failed to load and split data: {str(e)}")



In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Example inputs (replace with actual data)
sentence_input_data = "Example sentence text."
title_input_data = "Example title text."
abstract_input_data = "Example abstract text."
date_input_data = tf.constant([[1.0]])  # Example date input

# Tokenize inputs before passing them to the model
sentence_tokens = tokenizer(sentence_input_data, padding=True, truncation=True, return_tensors='tf')['input_ids']
title_tokens = tokenizer(title_input_data, padding=True, truncation=True, return_tensors='tf')['input_ids']
abstract_tokens = tokenizer(abstract_input_data, padding=True, truncation=True, return_tensors='tf')['input_ids']

# Encode the tokenized inputs with BERT
sentence_encoded = bert_model(sentence_tokens)[0]
title_encoded = bert_model(title_tokens)[0]
abstract_encoded = bert_model(abstract_tokens)[0]

# Expand dimensions of date_input_data to match the other encoded tensors
# Assuming that we want to concatenate along the last dimension:
date_input_expanded = tf.tile(date_input_data, [1, tf.shape(sentence_encoded)[1], tf.shape(sentence_encoded)[2]])

# Combine encoded text features with date
combined_features = tf.concat([sentence_encoded, title_encoded, abstract_encoded, date_input_expanded], axis=-1)

# Define output layer
output = Dense(1, activation='sigmoid')(combined_features)

# Define and compile the model
model = Model(inputs=[sentence_tokens, title_tokens, abstract_tokens, date_input_expanded], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Save model architecture and weights
model.save('project/models/citation_recommendation_model.h5')


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D, Concatenate, Dropout, LayerNormalization, MultiHeadAttention, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Define a custom Keras layer to wrap the BERT model
class BertLayer(Layer):
    def __init__(self, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert = bert_model

    def call(self, inputs):
        input_ids, attention_mask = inputs
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]
        return bert_output

# Define input layers
max_length = 128  # Adjust based on your data
input_ids = tf.keras.Input(shape=(max_length,), dtype='int32')
attention_mask = tf.keras.Input(shape=(max_length,), dtype='int32')
date_input = tf.keras.Input(shape=(1,), dtype='float32')

# Create instances of the BertLayer
sentence_encoder = BertLayer()(inputs=[input_ids, attention_mask])
title_encoder = BertLayer()(inputs=[input_ids, attention_mask])
abstract_encoder = BertLayer()(inputs=[input_ids, attention_mask])

# Function to encode text inputs - No changes are required here
def encode_text(input_ids, attention_mask):
    input_ids_tf = tf.cast(input_ids, tf.int32)
    attention_mask_tf = tf.cast(attention_mask, tf.int32)
    bert_output = bert_model(input_ids=input_ids_tf, attention_mask=attention_mask_tf)[0]
    return bert_output

# Encode the tokenized inputs with BERT - Use the BertLayer outputs instead of calling encode_text
sentence_encoded = sentence_encoder
title_encoded = title_encoder
abstract_encoded = abstract_encoder

# Apply attention mechanism
def attention_block(x, name):
    attn_output = MultiHeadAttention(num_heads=8, key_dim=64, name=f'{name}_attention')(x, x)
    attn_output = Dropout(0.1, name=f'{name}_attn_dropout')(attn_output)
    out1 = LayerNormalization(epsilon=1e-6, name=f'{name}_layernorm_1')(x + attn_output)
    ffn_output = Dense(512, activation='relu', name=f'{name}_ffn_1')(out1)
    ffn_output = Dense(768, name=f'{name}_ffn_2')(ffn_output)
    ffn_output = Dropout(0.1, name=f'{name}_ffn_dropout')(ffn_output)
    return LayerNormalization(epsilon=1e-6, name=f'{name}_layernorm_2')(out1 + ffn_output)

sentence_attn = attention_block(sentence_encoded, 'sentence')
title_attn = attention_block(title_encoded, 'title')
abstract_attn = attention_block(abstract_encoded, 'abstract')

# Apply global average pooling
sentence_pooled = GlobalAveragePooling1D(name='sentence_pooling')(sentence_attn)
title_pooled = GlobalAveragePooling1D(name='title_pooling')(title_attn)
abstract_pooled = GlobalAveragePooling1D(name='abstract_pooling')(abstract_attn)

# Process date input
date_features = Dense(64, activation='relu', name='date_embedding')(date_input)
date_features = LayerNormalization(epsilon=1e-6, name='date_layernorm')(date_features)

# Concatenate all features
combined_features = Concatenate(name='feature_concat')([sentence_pooled, title_pooled, abstract_pooled, date_features])

# Define hidden layers with residual connections
# Define hidden layers with residual connections
def residual_block(x, dropout_rate, name):
    residual = x
    units = x.shape[-1]  # Match units to the input feature dimension
    x = Dense(units, activation='relu', name=f'{name}_dense_1')(x)
    x = Dropout(dropout_rate, name=f'{name}_dropout_1')(x)
    x = Dense(units, name=f'{name}_dense_2')(x)
    x = Dropout(dropout_rate, name=f'{name}_dropout_2')(x)
    x = LayerNormalization(epsilon=1e-6, name=f'{name}_layernorm')(x + residual)
    return x

x = residual_block(combined_features, 0.2, 'residual_1')
x = residual_block(x, 0.2, 'residual_2')

# Define output layer
output = Dense(1, activation='sigmoid', name='output')(x)

# Define and compile the model
model = Model(inputs=[input_ids, attention_mask, date_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Example of how to use the model
def prepare_input(text):
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    return encoded['input_ids'], encoded['attention_mask']

sentence_input_ids, sentence_attention_mask = prepare_input("Example sentence text.")
title_input_ids, title_attention_mask = prepare_input("Example title text.")
abstract_input_ids, abstract_attention_mask = prepare_input("Example abstract text.")
date_input_data = tf.constant([[0.5]])  # Normalized date value

# Make a prediction
prediction = model.predict([sentence_input_ids, sentence_attention_mask, date_input_data])
print(f"Prediction: {prediction}")

# Save model architecture and weights
model.save('project/models/enhanced_citation_recommendation_model.h5')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer (BertLayer)    │ (None, 128, 768)       │              0 │ input_layer[0][0],     │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer_1 (BertLayer)  │ (None, 128, 768)       │              0 │ input_layer[0][0],     │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_layer_2 (BertLayer)  │ (None, 128, 768)       │              0 │ input_layer[0][0],     │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sentence_attention        │ (None, 128, 768)       │      1,575,168 │ bert_layer[0][0],      │
│ (MultiHeadAttention)      │                        │                │ bert_layer[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ title_attention           │ (None, 128, 768)       │      1,575,168 │ bert_layer_1[0][0],    │
│ (MultiHeadAttention)      │                        │                │ bert_layer_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ abstract_attention        │ (None, 128, 768)       │      1,575,168 │ bert_layer_2[0][0],    │
│ (MultiHeadAttention)      │                        │                │ bert_layer_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sentence_attn_dropout     │ (None, 128, 768)       │              0 │ sentence_attention[0]… │
│ (Dropout)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ title_attn_dropout        │ (None, 128, 768)       │              0 │ title_attention[0][0]  │
│ (Dropout)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ abstract_attn_dropout     │ (None, 128, 768)       │              0 │ abstract_attention[0]… │
│ (Dropout)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_8 (Add)               │ (None, 128, 768)       │              0 │ bert_layer[0][0],      │
│                           │                        │                │ sentence_attn_dropout… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_10 (Add)              │ (None, 128, 768)       │              0 │ bert_layer_1[0][0],    │
│                           │                        │                │ title_attn_dropout[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_12 (Add)         

 Total params: 29,549,121 (112.72 MB)

 Trainable params: 29,549,121 (112.72 MB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step


Prediction: [[0.5891415]]


In [ ]:
# scripts/trainExplanationModel.py
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('t5-small')
model = TFAutoModelForSeq2SeqLM.from_pretrained('t5-small')

# Define inputs
context_input = Input(shape=(None,), dtype=tf.string, name='context')
citation_input = Input(shape=(None,), dtype=tf.string, name='citation')

# Tokenize inputs
context_encoded = tokenizer(context_input, padding=True, truncation=True, return_tensors='tf')['input_ids']
citation_encoded = tokenizer(citation_input, padding=True, truncation=True, return_tensors='tf')['input_ids']

# Generate output sequence
output_sequence = model(input_ids=context_encoded, decoder_input_ids=citation_encoded)['logits']

# Define the model
explanation_model = Model(inputs=[context_input, citation_input], outputs=output_sequence)

# Save model architecture and weights
explanation_model.save('project/models/citation_explanation_model.h5')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


NameError: name 'Input' is not defined

In [ ]:
# scripts/evaluateModels.py
import tensorflow as tf
from sklearn.metrics import accuracy_score

# Load models
citation_model = tf.keras.models.load_model('project/models/citation_recommendation_model.h5')
explanation_model = tf.keras.models.load_model('project/models/citation_explanation_model.h5')

# Evaluate models
def evaluate_model(model, test_data):
    predictions = model.predict(test_data)
    accuracy = accuracy_score(test_data.labels, predictions.round())
    print(f'Model Accuracy: {accuracy:.4f}')

# Assuming test_data is loaded
evaluate_model(citation_model, test_data)
evaluate_model(explanation_model, test_data)


In [ ]:
# scripts/runModel.py
import tensorflow as tf

# Load models
citation_model = tf.keras.models.load_model('project/models/citation_recommendation_model.h5')
explanation_model = tf.keras.models.load_model('project/models/citation_explanation_model.h5')

# Example usage function
def recommend_and_explain(sentence, title, abstract):
    citation_prob = citation_model.predict([sentence, title, abstract])
    explanation = explanation_model.generate([sentence, title, abstract])
    return citation_prob, explanation

if __name__ == "__main__":
    sentence = "Your input sentence requiring citation."
    title = "Title of a potential reference article."
    abstract = "Abstract of a potential reference article."

    citation_prob, explanation = recommend_and_explain(sentence, title, abstract)
    print(f"Citation Probability: {citation_prob}")
    print(f"Explanation: {explanation}")
